In [1]:
# Installing Centaurminer chrome driver
!pip install centaurMiner==0.0.8
# Colab only: This is normally done automatically
!apt-get update # update ubuntu to correctly run apt-install
!apt install chromium-chromedriver # Installs to '/usr/lib/chromium-browser/chromedriver'
import time
import pandas_gbq
import pandas as pd
from datetime import datetime

     |████████████████████████████████| 911kB 10.0MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://d

In [2]:
import centaurminer as mining

In [ ]:
from google.colab import auth      # Google Authentication
auth.authenticate_user()
print('Authenticated')

Authenticated


In [10]:
project_id='for-yr'
table_id=f'Medical_Dataset.arxiv'
schema = [
    {'name': 'article_url', 'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'catalog_url', 'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'is_pdf',      'type': 'INT64',    'mode': 'REQUIRED'},
    {'name': 'language',    'type': 'STRING'                      },
    {'name': 'status',      'type': 'STRING',   'mode': 'REQUIRED'},
    {'name': 'timestamp',   'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'worker_id',   'type': 'STRING'                      },
    {'name': 'meta_info',   'type': 'STRING'                      }
]

In [13]:
class URL_builder():
  """ 
        Class to get the urls from the site with a search_word. 
  
        Parameters:
          search_word: word to be searched for urls. 
          * If the search_word consists of two words like social distancing,
            use +AND+Social+distancing
            Sample link would look this:
                      http://export.arxiv.org/find/all/1/all:+AND+Social+distancing/0/1/0/all/0/1?skip=0&query_id=c292936257c7949e

          
          pages: Number of pages from which the urls need to extracted.
          * Each page consists of only 25 articles.
  """
  def __init__(self,search_word,pages):
    """ 
        Class to get the urls from the site with a search_word. 
  
        Parameters:
          search_word: word to be searched for urls. 
          * If the search_word consists of two words like social distancing,
            use +AND+Social+distancing
            Sample link would look this:
                      http://export.arxiv.org/find/all/1/all:+AND+Social+distancing/0/1/0/all/0/1?skip=0&query_id=c292936257c7949e

          
          pages: Number of pages from which the urls need to extracted.
          * Each page consists of only 25 articles.
    """
    driver_path = '/usr/lib/chromium-browser/chromedriver'
    url_search=f"http://export.arxiv.org/find/all/1/all:+{search_word}/0/1/0/all/0/1?skip=".format(search_word)
    article_set = set()
    self.url1=[]
    self.url_schema=pd.DataFrame()
    for page_num in range(0,25*(pages),25):
      print(f"Getting URLs from {page_num + 1}...", flush=True)
      url=url_search+str(page_num)#+f"&query_id={query_id}".format(query_id)
      articles = mining.Element("css_selector","span.list-identifier > a").get_attribute('href')
      urls = mining.CollectURLs(url, articles,maxElems=20, driver_path=driver_path)
      [self.url1.append(i) for i in urls if 'abs' in i]
      self.url_schema['article_url']= self.url1
      self.url_schema['catalog_url']=url
      [article_set.add(i) for i in urls if 'abs' in i]
    self.url_schema['is_pdf']=0
    self.url_schema['language']='en'
    self.url_schema['status']='not mined'
    self.url_schema['timestamp']= datetime.utcnow()
    self.url_schema['worker_id']=None
    self.url_schema['meta_info']=None
    print("Total of " + str(len(article_set)) + " articles")
  

  def bigquery(self,project_id,table_id,schema):

    """ 
        Function to push the urls to the bigquery. 
  
        Parameters:
          project_id: project_id in bigquery. 
          table_id: table name.
          schema: schema to be followed.
    """ 
    # Append the new urls to the existing table
    pandas_gbq.to_gbq(self.new_urls, table_id, project_id=project_id, if_exists='append', table_schema=schema)
    return self.url_schema

In [14]:
urls=URL_builder('coronavirus',1)
urls.bigquery(project_id,table_id,schema)

Getting URLs from 1...
Headless: True
driver path: /usr/lib/chromium-browser/chromedriver
Total of 25 articles


,article_url,catalog_url,is_pdf,language,status,timestamp,worker_id,meta_info
0,http://export.arxiv.org/abs/2009.13577,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
1,http://export.arxiv.org/abs/2009.12923,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
2,http://export.arxiv.org/abs/2009.12817,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
3,http://export.arxiv.org/abs/2009.12698,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
4,http://export.arxiv.org/abs/2009.12500,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
5,http://export.arxiv.org/abs/2009.12176,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
6,http://export.arxiv.org/abs/2009.12076,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
7,http://export.arxiv.org/abs/2009.11714,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
8,http://export.arxiv.org/abs/2009.11288,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
9,http://export.arxiv.org/abs/2009.11008,http://export.arxiv.org/find/all/1/all:+corona...,0,en,not mined,2020-09-30 14:39:45.597776,None,None
